### Install Libraries

In [1]:
!pip install -Uqq pypdfium2
!pip install -Uqq tabula-py
!pip install -Uqq pdfplumber
!pip install -Uqqq python-dateutil

### Load Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pypdfium2 as pdfium
import pandas as pd
import numpy as np
import os.path
import tabula
from tabula.io import read_pdf
import pdfplumber
import re

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


### Create functions

In [3]:
# This function takes pdf as an input
# converts into text
# returns the required fields
    
def TeraNovaText(folder,file):
    pdf_fium = pdfium.PdfDocument(folder+'/'+file) # Read the PDF as a text File
    pdf_plumb = pdfplumber.open(folder+'/'+file) # Read the PDF as a text File
    
    no_of_pages = len(pdf_fium)
    
    out= []
    
    for i in range(no_of_pages):
        temp_out = []
        
        page_fium = pdf_fium[i] # pdfium
        page_plumb = pdf_plumb.pages[i] # PDF Plumber
        
        # Load a text page helper
        textpage_fium = page_fium.get_textpage() # pdfium
    
        # Extract text from the whole page
        text_fium = textpage_fium.get_text_range() # pdfium
        text_plumb = page_plumb.extract_text() # PDF Plumber
    
        # Split the list with the delimiters
        list_fium = text_fium.split('\r\n') # pdfium
        list_plumb = text_plumb.split('\n')  # PDF Plumber
    
    
        # Store the requird fields
    
        # Data Ingestion_datetime
        Ingestion_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
    
        # City
        City = list_plumb[3].split(',')[0].split(' ')[-1]
    
        # DATE MOLDED
        date_molded_src = "DATE MOLDED"
        date_molded_src_get_string = [x for x in list_fium if date_molded_src in x]
        date_molded_src_lst_str = ''.join(date_molded_src_get_string)
        Date_Molded = date_molded_src_lst_str.split(' ')[2]
    
        # DATE ISSUED
        date_issue_src = "DATE ISSUED"
        date_issue_src_get_string = [x for x in list_fium if date_issue_src in x]
        date_issue_src_lst_str = ''.join(date_issue_src_get_string)
        Date_Issued = date_issue_src_lst_str.split(':')[1].strip()
    
        # LAB NUMBER
        Lab_Number = date_molded_src_lst_str.split(' ')[5]
    
        # LOCATION OF PLACEMENT
        colon = ":"
        lop_searc = "LOCATION OF PLACEMENT"
        lop_searc_get_string = [x for x in list_fium if lop_searc in x]
        lop_searc_lst_str = ''.join(lop_searc_get_string)

        loc = lop_searc_lst_str

        if any(c in colon for c in loc):
            Location_of_Placement = loc.split(':')[1].strip()
        else:
            loc = loc.split(' ')
            loc.insert(3, ':')
            loc = " ".join(loc) 
            Location_of_Placement = loc.split(':')[1].strip()
        
        # CONCRETE SUPPLIER
        supplier_src = "CONCRETE SUPPLIER"
        supplier_src_get_string = [x for x in list_plumb if supplier_src in x]
        supplier_src_lst_str = ''.join(supplier_src_get_string)
        Concrete_Supplier = supplier_src_lst_str.split('WATER')[0].split(':')[1].strip()
    
        # MIX ID NO
        mixid_src = "MIX ID NO"
        mixid_src_get_string = [x for x in list_plumb if mixid_src in x]
        mixid_src_lst_str = ''.join(mixid_src_get_string)
        Mix_ID_No = mixid_src_lst_str.split('UNIT')[0].split(':')[1].strip()
    
        # WEATHER
        weather_src = "WEATHER"
        weather_src_get_string = [x for x in list_fium if weather_src in x]
        if weather_src_get_string:
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
            
        else:
            weather_src = "W EATHER"
            weather_src_get_string = [x for x in list_fium if weather_src in x]
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
    
        # TIME MOLDED
        time_src = "TIME MOLDED"
        time_src_get_string = [x for x in list_plumb if time_src in x]
        time_src_lst_str = ''.join(time_src_get_string)
        Time_Molded = time_src_lst_str.split(' ')[2] + ' ' +time_src_lst_str.split(' ')[3]
        if "ASTM" in Time_Molded:
            Time_Molded = ""
        else:
            Time_Molded = Time_Molded[0:8]
    
        # AIR CONTENT(%)
        aircontent_src = "AIR CONTENT"
        aircontent_src_get_string = [x for x in list_fium if aircontent_src in x]
        aircontent_src_lst_str = ''.join(aircontent_src_get_string)
        Air_Content = aircontent_src_lst_str.split(':')[1].strip()
        Air_Content = re.sub('[^0-9,.]', '', Air_Content)
    
        # SLUMP(IN)
        slump_src = "SLUMP(IN)"
        slump_src_get_string = [x for x in list_fium if slump_src in x]
        slump_src_lst_str = ''.join(slump_src_get_string)
        Slump_space_chck = slump_src_lst_str.split(':')
        if len(Slump_space_chck[1])>5:
            Slump = Slump_space_chck[1].split(' ')[1].strip()
        else:
            Slump = Slump_space_chck[1].strip()
        
        Slump = re.sub('[^0-9,.]', '', Slump)
        
    
        # SIZE & REQUIRED PSI 
        size_psi_search = "SIZE:"
        size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
        if size_psi_get_string:
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split(':')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()
        else:
            size_psi_search = "SIZE"
            size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split('SIZE')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()       
        
        
        # WATER ADDED(GALS)    
        water_searc = "GALS"
        water_searc_get_string = [x for x in list_fium if water_searc in x]
        water_searc_lst_str = ''.join(water_searc_get_string)
        Water_Added_len_check = water_searc_lst_str.split(':')
        
        if len(Water_Added_len_check[1])>5:
            Water_Added = Water_Added_len_check[1].split(' ')[1].strip()
        else:
            Water_Added = Water_Added_len_check[1].strip()
        
        Water_Added = re.sub('[^0-9,.]', '', Water_Added)
    
    
        # UNIT WEIGHT(PCF):
        unit_searc = "PCF"
        unit_searc_get_string = [x for x in list_fium if unit_searc in x]
        unit_searc_lst_str = ''.join(unit_searc_get_string)
        Unit_Weight = unit_searc_lst_str.split(':')[1].strip()
        Unit_Weight = re.sub('[^0-9,.]', '', Unit_Weight)
    
        # AMBIENT TEMP(F):
        ambi_temp_searc = "AMBIENT TEMP"
        ambi_temp_searc_get_string = [x for x in list_fium if ambi_temp_searc in x]
        ambi_temp_searc_lst_str = ''.join(ambi_temp_searc_get_string)
        Ambient_Temp = ambi_temp_searc_lst_str.split(':')[1].strip()
        Ambient_Temp = re.sub('[^0-9]', '', Ambient_Temp)
    
        # CONCRETE TEMP(F):
        concrete_temp_searc = "CONCRETE TEMP"
        concrete_temp_searc_get_string = [x for x in list_fium if concrete_temp_searc in x]
        concrete_temp_searc_lst_str = ''.join(concrete_temp_searc_get_string)
        Concrete_Temp = concrete_temp_searc_lst_str.split(':')[1].strip()
        Concrete_Temp = re.sub('[^0-9]', '', Concrete_Temp)
        
        temp_out = [file,Ingestion_datetime,City,Date_Molded,Date_Issued,Lab_Number,Location_of_Placement
       ,Concrete_Supplier,Mix_ID_No,Weather,Time_Molded,Air_Content,Slump,Size
       ,Required_PSI,Water_Added,Unit_Weight,Ambient_Temp,Concrete_Temp]
        
        out.append((temp_out))
    
    return out



In [4]:
# This function takes pdf as an input
# converts into a dataframe
# returns the test results as a DataFrame

def TeraNovaPdf2Table(folder,file,result):
    readpdf2df = tabula.io.read_pdf(folder+'/'+file 
                             , pages='all'
                             #,output_format="dataframe"
                                   , multiple_tables=True)
    
    df = pd.concat(readpdf2df)
    age_load_read = df.iloc[1:,3].dropna().reset_index().drop(columns=['index']).iloc[:-1,]
    
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('diameter', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('compression', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('age', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    
    # Extra Space in the Data Check
    space_check = age_load_read.iloc[:,0].str.split(' ').tolist()
    
    space_check_ref = []
    for i in space_check:
        for j in i:
            if len(i)>4:
                i[1]=i[1]+i[2]
                i.pop(2)
        space_check_ref.append(i)
    space_check = space_check_ref
    age_load_read = pd.DataFrame(space_check, columns =['Age_Days','Date_Tested','Total_load', 'Unit_load'])
    final_df = pd.DataFrame()
    for i in range(len(result)):
        df_split = np.array_split(age_load_read, len(result))
        
        result_lab = result[i][5]
        result_date_isssue = result[i][4]
        
        df_sel = df_split[i]
        
        iterat = df_sel.shape[0]
        
        lab_nbr = pd.DataFrame({'Lab_Number': result_lab}, index=[0])
        date_isu = pd.DataFrame({'Date_Issued': result_date_isssue}, index=[0])
        
        lab_nbr_itr = pd.DataFrame(np.repeat(lab_nbr.values, iterat, axis=0))
        date_isu_itr = pd.DataFrame(np.repeat(date_isu.values, iterat, axis=0))
        
        lab_nbr_itr.columns = lab_nbr.columns
        date_isu_itr.columns = date_isu.columns
        
        temp_df = pd.concat([lab_nbr_itr,date_isu_itr,age_load_read.iloc[:, age_load_read.columns != 'Total_load']], axis=1, join='inner')
        final_df = final_df.append(temp_df).reset_index(drop=True)
        
    
    return final_df
    
    
    

### Main Function.  Make sure to change the path of the testing folder befor running it.

In [5]:
log = []
df_text = []
pdf_df = pd.DataFrame()

#path = '/Users/prateek/Documents/Captsone Project/Jupiter Files/TestFolder'

os.chdir('/Users/prateek/Documents/Captsone Project/Test Folder')
path = '/Users/prateek/Documents/Captsone Project/Test Folder/sample tera nova test'

for file in os.listdir(path+'/'):
    if file.endswith('.pdf'):
        read_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
        folder = os.path.basename(path)
        dir_path = os.path.dirname(path)
        
        log.append((dir_path,folder,file,read_datetime))  # Appends results for log DF
        
        result = TeraNovaText(folder,file) # Calls text function
        for j in range(len(result)):
            df_text.append((result[j]))    # Appends results from text function
        
        df_out = TeraNovaPdf2Table(folder,file,result) # Calls Dataframe function
        pdf_df = pdf_df.append(df_out).reset_index(drop=True) # Appends test results DF
        
        
          

# Append Log DataFrame        
df_log = pd.DataFrame(log, columns=['Directory','Folder','File', 'Read_DateTime'])

# Append DataFrame with Text fields
df_page = pd.DataFrame(df_text, columns=['File_Name',
                     'Ingestion_datetime',
                     'City',
                     'Date_Molded',
                     'Date_Issued',
                     'Lab_Number',
                     'Location_of_Placement',
                     'Concrete_Supplier',
                     'Mix_ID_No',
                     'Weather',
                     'Time_Molded',
                     'Air_Content(%)',
                     'Slump(in)',
                     'Size',
                     'Required_PSI',
                     'Water_Added(GALS)',
                     'Unit_Weight(PCF)',
                     'Ambient_Temp(F)',
                     'Concrete_Temp(F)'])




In [6]:
#log

In [7]:
#result

In [8]:
#df_text

In [9]:
#df_log

In [10]:
#df_page

In [11]:
#pdf_df

### Save the Data in Excel Files

In [12]:
df_page.to_excel('COMPRESSION_TEST_SPECIMENS_REPORT.xlsx', index=False)
pdf_df.to_excel('COMPRESSION_TEST_RESULTS.xlsx', index=False)
df_log.to_excel('INGESTION_LOG.xlsx', index=False)

files_write_dt = datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')


### Job Log

In [13]:
ends = datetime.strptime(df_page['Ingestion_datetime'].iloc[-1], '%m/%d/%Y %I:%M:%S %p')
start = datetime.strptime(df_log['Read_DateTime'][0], '%m/%d/%Y %I:%M:%S %p')

write_end = datetime.strptime(files_write_dt, '%m/%d/%Y %I:%M:%S %p')

diff1 = relativedelta(ends,start)
diff2 = relativedelta(write_end,start)


In [14]:
with open('Joblog.txt', 'w') as f:
    print('=============================================', file=f)
    print('  ',date.today().strftime("%A,%d %B, %Y"), file=f)
    print('\t',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
    print('=============================================', file=f)
    print("TeraNova Scrapper Job Completed Successfully", file=f)
    print('=============================================', file=f)
    print('Total Files Read:', len(df_log), file=f)
    print('Total Pages Ingested:', len(df_page), file=f)
    print('=============================================', file=f)
    print('First File Read Date Time:',df_log['Read_DateTime'][0], file=f)
    print('Last File Ingestion Data Time:',df_page['Ingestion_datetime'].iloc[-1], file=f)
    print('=============================================', file=f)
    print("Reading to Ingesting files took %d hours %d minutes %d seconds" % (diff1.hours, diff1.minutes, diff1.seconds)
          , file=f)
    print('=============================================', file=f)
    print("Total job took %d hours %d minutes %d seconds" % (diff2.hours, diff2.minutes, diff2.seconds)
         , file=f)
    print('=============================================', file=f)
    
    
    

In [15]:
print("Check Job Log File!!")

Check Job Log File!!


### Data Transformation

In [44]:
# Create a copy of the dataframes
df_page_stg = df_page.copy()
pdf_df_stg = pdf_df.copy()

# Rename the Columns
df_page_stg = df_page_stg.rename(columns={'File_Name': 'ReportFileName'
                                          , 'Date_Issued': 'ReportDateIssued'
                                          , 'Ingestion_datetime': 'ReportTimeIngested'
                                          , 'Ambient_Temp(F)': 'SiteTemperature'
                                          , 'Weather': 'SiteWeather'
                                          , 'Concrete_Supplier': 'ConcreteSupplier'
                                          , 'Location_of_Placement': 'ConcretePlacementLocation'
                                          , 'Lab_Number': 'BatchLabNumber'
                                          , 'Mix_ID_No': 'BatchMixID'
                                          , 'Size': 'BatchSpecimenSize'
                                          , 'Concrete_Temp(F)': 'BatchTemperature'
                                          , 'Unit_Weight(PCF)': 'BatchUnitWeight'
                                          , 'Required_PSI': 'BatchRequiredStrength'
                                          , 'Air_Content(%)': 'BatchAirContent'
                                          , 'Slump(in)': 'BatchtSlump'
                                          , 'Water_Added(GALS)': 'BatchWaterAdded'})



pdf_df_stg = pdf_df_stg.rename(columns={'Lab_Number': 'BatchLabNumber'
                                          , 'Date_Issued': 'ReportDateIssued'
                                          , 'Date_Tested': 'SpecimenDateTested'
                                          , 'Unit_load': 'SpecimenMeasuredStrength'
                                          , 'Age_Days': 'SpecimentAgeTested'})



In [45]:
########### Changing Data Types of Columns for SpecimenFieldReport #########

df_page_stg['ReportTimeIngested'] = df_page_stg['ReportTimeIngested'].astype('datetime64[ns]')
df_page_stg['ReportDateIssued'] = pd.to_datetime(df_page_stg["ReportDateIssued"], format="%d-%b-%y")

df_page_stg['SiteTemperature'] = df_page_stg['SiteTemperature'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchTemperature'] = df_page_stg['BatchTemperature'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchUnitWeight'] = df_page_stg['BatchUnitWeight'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchRequiredStrength'] = df_page_stg['BatchRequiredStrength'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchAirContent'] = df_page_stg['BatchAirContent'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchtSlump'] = df_page_stg['BatchtSlump'].replace(r'^\s*$', np.nan, regex=True)
df_page_stg['BatchWaterAdded'] = df_page_stg['BatchWaterAdded'].replace(r'^\s*$', np.nan, regex=True)

df_page_stg['BatchLabNumber'] = df_page_stg['BatchLabNumber'].astype('int')
df_page_stg['SiteTemperature'] = df_page_stg['SiteTemperature'].astype('float')
df_page_stg['BatchTemperature'] = df_page_stg['BatchTemperature'].astype('float')
df_page_stg['BatchUnitWeight'] = df_page_stg['BatchUnitWeight'].astype('float')
df_page_stg['BatchRequiredStrength'] = df_page_stg['BatchRequiredStrength'].astype('float')
df_page_stg['BatchAirContent'] = df_page_stg['BatchAirContent'].astype('float')
df_page_stg['BatchtSlump'] = df_page_stg['BatchtSlump'].astype('float')
df_page_stg['BatchWaterAdded'] = df_page_stg['BatchWaterAdded'].astype('float')

df_page_stg["Time_Molded"] = df_page_stg['Time_Molded'].str.lower()
df_page_stg['Time_Molded'] = df_page_stg['Time_Molded'].replace({'sm':'am', 'om':'pm'}, regex = True)

df_page_stg["BatchTimeMolded"] = df_page_stg["Date_Molded"] + ' ' +df_page_stg["Time_Molded"]
df_page_stg['BatchTimeMolded'] = df_page_stg['BatchTimeMolded'].astype('datetime64[ns]')



########### Changing Data Types of Columns for SpecimenCompressionTestResult #########

pdf_df_stg['ReportDateIssued'] = pd.to_datetime(pdf_df_stg["ReportDateIssued"], format="%d-%b-%y")

junk_df = pd.DataFrame()


for i in pdf_df_stg['SpecimenDateTested']:
    if len(i)!= 9:
        junk_df = junk_df.append(pdf_df_stg.loc[pdf_df_stg['SpecimenDateTested'] == i]).reset_index(drop=True)
        pdf_df_stg = pdf_df_stg.drop(labels=[pdf_df_stg.loc[pdf_df_stg['SpecimenDateTested'] == i].index.values[0]]
                                     , axis=0).reset_index(drop=True)

pdf_df_stg['SpecimenDateTested'] = pd.to_datetime(pdf_df_stg["SpecimenDateTested"], format="%d-%b-%y")

if junk_df.empty:
    pass
else:
    junk_df['SpecimenDateTested'] = pd.to_datetime(junk_df["SpecimenDateTested"], format="%d-%m-%y")
    pdf_df_stg = pd.concat([pdf_df_stg, junk_df], ignore_index=True, sort=False)

del junk_df


pdf_df_stg['BatchLabNumber'] = pdf_df_stg['BatchLabNumber'].astype('int')
#pdf_df_stg['SpecimentAgeTested'] = pdf_df_stg['SpecimentAgeTested'].astype('int')

In [56]:
# pdf_df_stg.dtypes

In [55]:
pdf_df_stg_new  = pdf_df_stg[pdf_df_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == pdf_df_stg['ReportDateIssued']].reset_index(drop=True)


#pdf_df_stg_new



In [53]:
#pdf_df_stg_new['SpecimentAgeTested'].unique()

In [54]:
df_page_stg_new  = df_page_stg[df_page_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == df_page_stg['ReportDateIssued']].reset_index(drop=True)


#df_page_stg_new


In [52]:
df_page_stg_new.to_excel('SpecimenFieldReport.xlsx', index=False)
pdf_df_stg_new.to_excel('SpecimenCompressionTestResult.xlsx', index=False)

In [ ]:
print("Success!! Check SpecimenFieldReport and SpecimenCompressionTestResult files")